In [1]:
# prototyping a Python script to persist prediction results on the database
# per DMSLSTM model, per test dataset

In [2]:
import json
import numpy as np
import pandas as pd

In [13]:
# arguments to parse:
# model identifier in the stats/ folder
# json file per dataset to test on

In [14]:
model_id = 'DMSLSTM_TPU_70_00'

In [16]:
dataset = 'test'

if source == 'gcp':
    prefix = 'gs://'
elif source == 'local':
    prefix = '/home/developer/gcp'

In [ ]:
# prepare the script for local execution only

In [20]:
path = '/home/developer/gcp/cbidmltsf/stats/{}/prediction_results_on_{}_tfrecord.json'.format(model_id, dataset)

In [21]:
path

'/home/developer/gcp/cbidmltsf/stats/DMSLSTM_TPU_70_00/prediction_results_on_test_tfrecord.json'

In [22]:
# build a Python dict from the json file
with open(path, 'r') as json_file:
    predictions_dict = json.load(json_file)

In [27]:
columns=['model_id', 'dataset', 'string_timestamp', 'prediction', 'target']

In [28]:
# how many predictions in the dataset?
length = len(predictions_dict['predictions'])
# a list with model_id repeated length times
model_id_repeat_list = [model_id]*length
# same for dataset
dataset_repeat_list = [dataset]*length
# predictions dataframe
predictions_df = pd.DataFrame(list(zip(model_id_repeat_list,
                                       dataset_repeat_list,
                                       predictions_dict['string_timestamps'],
                                       predictions_dict['predictions'],
                                       predictions_dict['targets'])), columns = columns)

In [29]:
predictions_df

,model_id,dataset,string_timestamp,prediction,target
0,DMSLSTM_TPU_70_00,test,2018-02-23 14:00:00,8.744109,6.213650
1,DMSLSTM_TPU_70_00,test,2018-02-23 15:00:00,8.350501,8.161867
2,DMSLSTM_TPU_70_00,test,2018-02-23 16:00:00,8.017948,6.752000
3,DMSLSTM_TPU_70_00,test,2018-02-23 17:00:00,4.895577,5.244833
4,DMSLSTM_TPU_70_00,test,2018-02-23 18:00:00,6.951962,7.247250
...,...,...,...,...,...
125,DMSLSTM_TPU_70_00,test,2018-02-28 19:00:00,8.483326,9.289534
126,DMSLSTM_TPU_70_00,test,2018-02-28 20:00:00,8.870011,9.598534
127,DMSLSTM_TPU_70_00,test,2018-02-28 21:00:00,8.932780,6.592383
128,DMSLSTM_TPU_70_00,test,2018-02-28 22:00:00,5.522929,6.462183


In [30]:
# build a path to persist the dataframe to database/
# /home/developer/gcp/cbidmltsf/database/predictions/DMSLSTM_TPU_10_00_on_test_tfrecord.pkl

In [32]:
pickle_path = '/home/developer/gcp/cbidmltsf/database/predictions/{}_on_{}_tfrecord.pkl'.format(model_id, dataset)

In [34]:
# persist the Pandas dataframe to database/
predictions_df.to_pickle(pickle_path)

In [36]:
# verify the persisted dataframe
df = pd.read_pickle(pickle_path)

In [37]:
df

,model_id,dataset,string_timestamp,prediction,target
0,DMSLSTM_TPU_70_00,test,2018-02-23 14:00:00,8.744109,6.213650
1,DMSLSTM_TPU_70_00,test,2018-02-23 15:00:00,8.350501,8.161867
2,DMSLSTM_TPU_70_00,test,2018-02-23 16:00:00,8.017948,6.752000
3,DMSLSTM_TPU_70_00,test,2018-02-23 17:00:00,4.895577,5.244833
4,DMSLSTM_TPU_70_00,test,2018-02-23 18:00:00,6.951962,7.247250
...,...,...,...,...,...
125,DMSLSTM_TPU_70_00,test,2018-02-28 19:00:00,8.483326,9.289534
126,DMSLSTM_TPU_70_00,test,2018-02-28 20:00:00,8.870011,9.598534
127,DMSLSTM_TPU_70_00,test,2018-02-28 21:00:00,8.932780,6.592383
128,DMSLSTM_TPU_70_00,test,2018-02-28 22:00:00,5.522929,6.462183


In [38]:
# now make a script from the previous code